In [2]:
from assignment2 import *

pd.options.display.float_format = "{:,.4f}".format

Thomas Vaudescal : 11237578

William Delisle : 11313171

Sébastien Caron : 11271997

In [3]:
days_in_year = 252
n_days = 10 * days_in_year
horizon = np.arange(1, n_days + 2) / days_in_year

In [4]:
log_xreturns = get_log_excess_returns(days_in_year)
log_xreturns

date
1996-01-02       NaN
1996-01-03    0.0007
1996-01-04   -0.0061
1996-01-05   -0.0018
1996-01-08    0.0026
               ...  
2021-12-27    0.0137
2021-12-28   -0.0010
2021-12-29    0.0014
2021-12-30   -0.0030
2021-12-31   -0.0027
Name: log_xret, Length: 6547, dtype: float64

Utilisez les paramètres initiaux suivants pour démarrer l'estimation des paramètres d'un modèle NGARCH en date du 1996-12-31. Utilisez la notation suivante:

\begin{align*}
r_{t+1} - r_{f} &= \lambda \sqrt{h_{t+1}}-\frac{1}{2}h_{t+1}+\sqrt{h_{t+1}}%
\varepsilon _{t+1}, \\
h_{t+1}& =\omega +\alpha h_{t}\left( \varepsilon _{t}-\gamma \right)
^{2}+\beta h_{t}
\end{align*}

Quels paramètres obtenez vous en maximisant la vraisemblance des rendements au 1996-12-31? Répétez l'exercice au 2020-02-01. Tabulez les paramètres de façon à pouvoir comparer les deux jeux de paramètres aisément. Ajoutez à votre table 

- la volatilité inconditionnelle (annualisée)
- la volatilité conditionnelle (annualisée) prédite au lendemain de l'estimation
- la correlation conditionnelle en $t$ entre $r_{t+1}$ et $h_{t+2}$
- la volatilité conditionnelle en $t$ de la variance $h_{t+2}$

que les paramètres impliques. De plus, inspirez-vous de la `note_on_erp.ipynb` et définissez une fonction:
```
def plot_var_forecasts(horizon, P, Q, annualized=False):
```
que vous appelerez avec `annualized=True` sur des simulations résultant des deux différents jeux de paramètres. Discutez des paramètres, des figures et des implications aux point de vue de la validité de la spécification (vs *misspecification*) du NGARCH.

In [5]:
# Call the function for the misspecified model (non-optimized)
out_Q1_m, P_1996_m, P_2020_m, Q_1996_m, Q_2020_m, ng1996_m, ng2020_m = compute_model_Q1(
    log_xreturns=log_xreturns,
    days_in_year=days_in_year,
    horizon=horizon,
    f_ngarch=f_ngarch,
    measure=measure,
    f_out_format_Q1=f_out_format_Q1,
    misspecified=True
)
# Misspecified Model Output
print("\nMisspecified Model Output:")
out_Q1_m

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
plot_var_forecasts2(horizon, P_1996_m, Q_1996_m, annualized=True)

In [ ]:
# Call the function for the specified model (optimized)
out_Q1, P_1996, P_2020, Q_1996, Q_2020, ng1996, ng2020 = compute_model_Q1(
    log_xreturns=log_xreturns,
    days_in_year=days_in_year,
    horizon=horizon,
    f_ngarch=f_ngarch,
    measure=measure,
    f_out_format_Q1=f_out_format_Q1,
    misspecified=False
)
# Specified Model Output
print("Specified Model Output:")
out_Q1

In [ ]:
plot_var_forecasts2(horizon, P_1996, Q_1996, annualized=True)

In [ ]:
print(
    f" Est-ce que les contraintes sur les paramètres et la condition de stationnarité sont satisfaites ? : "
    f"{check_constraints_and_stationarity(out_Q1, bounds=[(0, 10), (0, 10), (0, 1), (0.5, 1), (0, 10)])}"
)                                                      #   lambda,   omega,   alpha,   gamma,    beta

- Au niveau des paramètres, on remarque ci-dessus que toutes les contraintes sont satisfaites, dont la contrainte de stationnarité : $\alpha (1 + \gamma^2) + \beta < 1$.

La volatilité conditionnelle sous Q est généralement supérieure à celle sous P en raison de l'aversion au risque des investisseurs. Les investisseurs exigent une prime de risque pour détenir des actifs risqués et cette prime est intégrée dans la tarification des produits dérivés. La volatilité sous Q reflète cette prime de risque en étant plus élevée que la volatilité sous P, car les investisseurs anticipent un niveau de risque plus élevé lorsqu'ils évaluent les produits dérivés (aussi appelée prime de volatilité).

# Question 2

Nous souhaitons tarifer les options aux deux dates pour lesquelles nous avons estimé les paramètres ci-haut. Avant de procédez, comparer qualitativement les deux jeux de données. Soutenez votre propos à l'aide de statistiques descriptives (tables) et de figures au besoin. Quels filtres devrait-on appliquer aux données avant procéder à la tarification des options? Justifier.

In [ ]:
# Construction de la table descriptive
option_info = read_and_concatenate_options(
    "opprcd961203_108105.csv", "opprcd200201_108105.csv"
)
spx = load_price()

f_describe_table(option_info, spx).dropna(axis=1)

La première étape, avant même de construire la table de statistiques descriptives, a été d'enlever les données pour lesquelles on a une volatilité implicite manquante et filtrer les lignes identiques. Ce sont les premiers filtres à appliquer aux données avant la tarification. Il est ensuite possible de comparer les deux jeux de données à l'aide de la table ci-dessus.

Le premier constat frappant est la différence entre le nombre d'option pour lesquels on a des données. En effet, il y en a plus de 43 fois plus en 2020 qu'en 1996. L'intervalle de prix d'exercice possible a aussi beaucoup augmenté. Par exemple, pour les puts, on voit que le prix d'exercice minimal est plus bas en 2020 qu'en 1996, et ce, malgré le fait que le prix du S&P500 a beaucoup augmenté depuis. Ce phénomène est aussi observable à travers l'augmentation des valeurs maximales de volatilité implicite entre 1996 et 2020. Finalement, il y a aussi un écart entre les maturités offertes. On peut observer que des options autant à plus courte qu'à plus longue échéance sont disponibles en 2020 par rapport à 1996.

Pour ce qui est des autres filtres à appliquer aux données avant la tarification, on doit premièrement s'assurer qu'il n'y a pas d'options avec une volatilité implicite extrême. En effet, une volatilité implicite extrême tend à ne pas représenter les vraies attentes des participants du marché. Nous fixerons à 125% la volatilité maximale pour laquelle on garde les données des options. Ensuite, on doit simplement s'assurer qu'il n'y a pas d'incohérence dans les données du delta (delta des calls entre 0 et 1 avec delta des puts entre -1 et 0). Le dernier filtre à appliquer sera sur le temps à maturité des options. Sachant que des options à très court terme peuvent se comporter de manière anormal, nous allons filtrer ceux avec une maturité de moins de 7 jours (une semaine). Nous allons aussi filtrer les options avec de très longue maturité car celles-ci manquent souvent de liquidité. La contrainte maximale pour la maturité sera de 550 jours (environ 1 an et demi).

Il reste maintenant à appliquer les filtrations décrites. Or, on s'apperçoit à l'aide de la table descriptive qu'il sera seulement nécessaire de filtrer les données de 2020 par rapport au temps restant avant la maturité. C'est ce qui est fait ci-dessous.


In [ ]:
# Filtration des données
option_info = f_clean_table(option_info, spx, keep_moneyness=True).reset_index(
    drop=True
)
option_info

In [ ]:
plot_iv_surface3d(option_info, date="1996-12-03")

In [ ]:
plot_iv_surface3d(option_info, date="2020-02-03")

# Question 3

Afin de tarifer les options, nous devons calculer, pour chaque paire $(t,T)$, le prix ex-dividendes du sous-jacent. Dans le cas du S&P 500, deux options s'offre à nous:

1. Utiliser en $t$ le taux de dividende prévu ($y$, en termes annuels) par OptionMetrics pour calculer $S(t) e^{-y(T-t)}$.

2. Utiliser l'approche du Cboe pour calculer le option-implied forward price $F(t,T)$ et la courbe des taux fournis par OptionMetrics (`zerocd.csv`). Ensuite, calculer le ex-dividend price $F(t,T) e^{-r_{t,T} (T-t)}$, où $r_{t,T}$ est le taux sans risque à $t$ pour une maturité à $T$, 

Comparer les résultats obtenus à les des deux approches. Comment et pourquoi diffèrent-ils selon vous?

In [ ]:
option_info = read_and_concatenate_options(
    "opprcd961203_108105.csv", "opprcd200201_108105.csv"
)
option_info = f_clean_table(option_info, spx).reset_index(drop=True)

In [ ]:
# Ajouter les prix, les dividendes, les rf et la moneyness à option_info
f_add_Q3_info(option_info, days_in_year)

# Méthode 1
exdiv_1 = option_info.S_t * np.exp(-option_info.y_t * (option_info.DTM) / days_in_year)
option_info["exdiv_1"] = exdiv_1

# Méthode 2
option_info = f_F_CBOE(option_info, days_in_year)
exdiv_2 = option_info.F_CBOE * np.exp(-option_info.r_f * (option_info.DTM) / days_in_year)
option_info["exdiv_2"] = exdiv_2

# Enlever les maturités pour lesquelles on ne peut pas calculer le forward CBOE
option_info.dropna(subset=["exdiv_2"], how="any", inplace=True)

# Graphique de comparaison
comp_plot = f_plot_Q3_comparison(option_info)

comparison_df = create_comparison_df(option_info)

In [ ]:
comparison_df

La première chose importante à noter est que, à cause des filtres appliqués à la question 2, on ne pouvait pas évaluer le option-implied forward price à l'aide de la méthode CBOE pour un nombre de jours à maturité de 179 (2020-02-03). En effet, il n'y avait pas au moins un prix d'exercice pour lequel on avait un call et un put à cette maturité. Les options pour cette maturité et pour la date d'évaluation du 2020-02-03 ne pourront donc malheureusement pas être analysées à cause de la nature des questions suivantes.

Pour ce qui est de la comparaison entre les deux méthodes, il est possible d'observer dans les graphiques précédents qu'il y a toujours une différence. Pour les données de 1996, la méthode 1 donne toujours un prix ex-dividende supérieur à la méthode 2. Pour ce qui est des données de 2020, c'est seulement vrai pour des maturités jusqu'à 74 jours. De plus, pour cette maturité, les prix donnés par les deux méthodes sont presque identiques. Pour des maturités plus grandes, c'est la méthode 2 qui donne des prix ex-dividendes plus élevés. Il est aussi intéressant de noter que pour de longues maturités, l'écart entre les deux méthodes s'accentuent (aautant en 2020 qu'en 1996).

D'ailleurs, le fait que les deux méthodes ne donne pas le même résultat pour le prix ex-dividende est logique. En effet, pour que ce soit le cas, il aurait fallu que pour chacune des maturités, il existe au moins un put ou un call pour lequel la moneyness(K/F) est exactement de 1. Cela aurait fait en sorte que l'approche du CBOE (méthode 2) coincide avec l'approche théorique (méthode 1). En d'autres mots, il aurait fallu une différence nulle entre le prix d'un put et d'un call de même strike pour que la méthode 2 coincide avec la méthode 1. Or, comme c'est jamais exactement le cas, il y a toujours une différence entre les deux méthodes.

# Question 4

Notez que sous BMS, un call
\begin{align*}
 c(t,T,K) &= \hat{S}(t) N(d_1) - Ke^{-r_f(T-t)} N(d_2) \\
% \textrm{avec} 
 d_i &= \frac{\log\frac{\hat{S}(t)}{K} + \left(r_f \pm \frac{1}{2}\sigma^2 \right) (T-t)
     }{\sigma\sqrt{T-t}}
\end{align*}
avec $\hat{S}(t) = S(t)e^{-y(T-t)} = F(t,T)e^{-r_f(T-t)}$. L'argument est similaire pour un put. On peut donc calculer les volatilités implicites de chaque option avec l'approche utilisée dans le TP1, avec $y=0$, mais en utilisant les prix ex-dividendes obtenus en Question 3. 

Comparer les 2 volatilités implicites ainsi obtenues avec celle fournie par OptionMetrics (`impl_volatility`). Illustrer votre comparaison. 

In [ ]:
# Calculer les IV pour les deux types de prix ex-dividende
impl_vol_mkt = bms.implied_volatility(
    opt_price=option_info.mean_bidask,
    S=[option_info.exdiv_1, option_info.exdiv_2],
    K=option_info.strike_price / 1000,
    r=option_info.r_f,
    y=0,
    T=option_info.DTM / days_in_year,
    is_call=(option_info.cp_flag == "C"),
)

# Ajouter les résultats à option_info
option_info["IV_method1"] = impl_vol_mkt[0]
option_info["IV_method2"] = impl_vol_mkt[1]

# Enlever les options pour lesquelles on a une IV négatif pour une des deux méthéodes
option_info = option_info[
    (option_info["IV_method1"] >= 0) & (option_info["IV_method2"] >= 0)
]

# Smile pour toutes les maturités (pour les deux dates)
date_unique = np.unique(option_info.date)
f_plot_Q4_smiles(option_info, date_unique[0], "1996-12-03")
f_plot_Q4_smiles(option_info, date_unique[1], "2020-02-03")

# Question 5

Dans la même optique, dénotons $\tilde{R}_{t,T}^{(j)} = \exp\left\lbrace\sum_{k=1}^{T-t} r_{t+k}^{(j)} - r_f\right\rbrace$ le rendement excédentaire à terme obtenus par simulation du modèle NGARCH. Notez que le payoff actualisé d'une option peut être écrit 
\begin{align*}
  \mathcal{O}^{(j)}_{i} = e^{-r_{t,T_i} (T_i-t)}\max\left\lbrace 0, cp_i\left(F_{t,T_i} \tilde{R}_{t,T_i}^{(j)} - K\right) \right\rbrace
\end{align*}
où $cp_i = 1$ pour un call $i$ ($cp_i = -1$ pour le put $i$) de maturité $T_i$ et de strike $K_i$. Ainsi, seul $\tilde{R}_{t,T}^{(j)}$ dépend de la trajectoire simulée. En $t$, en simulant une seule fois 100,000 trajectoires de longueur $T_{max} - t$, il est donc possible de tarifer toutes les options observées ce jour là.

Procéder ainsi en utilisant l'un où l'autre des prix forward considérés en Question 3. Comparer les IV obtenues en Question 4, selon les prix de marché, avec les IV obtenus en appliquant la même méthode numérique au prix NGARCH. En particulier, tabuler les erreurs de tarifications, dans le domaine de la IV, en fonction d'intervalles de maturité et de moneyness.

In [ ]:
# Simulation des returns avec le modèle GARCH pour plus grande maturité sous Q
option_info = simulate_returns(option_info, ng1996, ng2020, n_paths=100_000)

In [ ]:
option_info

In [ ]:
option_info = calculate_option_prices(option_info, ng1996, ng2020)

In [ ]:
option_info

In [ ]:
impl_vol_ng = bms.implied_volatility(
    opt_price=option_info.Option_price,
    S=option_info.exdiv_2,
    K=option_info.strike_price / 1000,
    r=option_info.r_f,
    y=0,
    T=option_info.DTM / days_in_year,
    is_call=(option_info.cp_flag == "C"),
)


In [ ]:
option_info["IV_NGARCH"] = impl_vol_ng
option_info[
    [
        "date",
        "exdate",
        "cp_flag",
        "strike_price",
        "impl_volatility",
        "IV_method1",
        "IV_method2",
        "IV_NGARCH",
    ]
]